In [2]:
# Imports
import pandas as pd
import autograd.numpy as np  # Thinly-wrapped numpy
from autograd import grad 
from autograd import elementwise_grad
from matplotlib import pyplot as plt
from numpy.random import multivariate_normal as N
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix

In [3]:
wFinal = np.array([  4.22455732e-02,   4.22473088e-02,   5.37630673e-06,
         4.22455732e-02,   5.37596623e-06,   4.22473088e-02,
         4.22455732e-02,   4.22473088e-02,   4.22473221e-02,
         4.22473222e-02,   4.22455732e-02,  -2.54292998e-06,
         4.22455696e-02,   4.22455732e-02,   5.37532906e-06,
         5.37634719e-06,  -5.21942825e-06,  -5.21937071e-06,
        -5.21942778e-06,   4.22455732e-02,   5.37634719e-06,
         5.37634675e-06,   5.37634719e-06,   4.22455696e-02,
         4.22473086e-02,   5.21942780e-06,   5.37533456e-06,
         4.22455732e-02,   5.37533405e-06,  -2.78179336e-06,
         4.22455732e-02,  -5.21942878e-06,   5.21229106e-06,
        -5.21943212e-06,  -2.78179336e-06,   4.22473222e-02,
         4.22455732e-02,   4.22455696e-02,   5.21229121e-06,
        -2.78179361e-06,   4.22455732e-02,  -2.40925210e-06,
        -5.21942778e-06,   5.37630675e-06,  -5.21942825e-06,
         4.22455696e-02,   4.22455696e-02,  -2.78179294e-06,
         4.22455696e-02,  -5.21942748e-06,   5.21229106e-06,
         5.37533445e-06,  -2.78179294e-06,  -5.21942748e-06,
         4.22455696e-02,   5.21229209e-06,  -5.21942878e-06,
         5.37533405e-06,  -2.78397019e-06,  -5.21942778e-06,
        -5.21942825e-06,   5.37533445e-06,  -5.21942748e-06,
        -5.21942778e-06,   4.22455732e-02,   4.22473086e-02,
        -5.21942748e-06,  -5.21942878e-06,  -5.21942778e-06,
         5.21942778e-06,   5.37533456e-06,   5.21220933e-06,
        -5.21943212e-06,  -2.78396471e-06,   5.21303725e-06,
         5.21229121e-06,   5.37532905e-06,  -2.40932524e-06,
         4.22455732e-02,  -5.21942778e-06,   4.22455696e-02,
         5.21942778e-06,   5.37532905e-06,  -5.21943155e-06,
         4.22455696e-02,   4.22455696e-02,  -5.21942778e-06,
         5.21229106e-06,   5.21942778e-06,   5.21942778e-06,
         5.21942778e-06,  -5.21942878e-06,   5.21942780e-06,
        -5.21942748e-06,   5.21942778e-06,   5.21942780e-06,
        -5.21942878e-06,   5.21229121e-06,  -5.21942878e-06,
         4.22455696e-02,   5.21942778e-06,   5.21942778e-06,
         5.21229209e-06,  -5.21942778e-06,   4.22455696e-02,
         5.21942778e-06,   5.37532905e-06,  -5.21942748e-06,
        -5.21942878e-06,   5.21229112e-06,   5.21942778e-06,
        -5.21942778e-06,   5.21229106e-06,   4.22455696e-02,
         5.21942778e-06,  -5.21942778e-06,   5.21942778e-06,
        -5.21943210e-06,  -5.21942878e-06,   4.22455696e-02,
        -5.21936489e-06,   5.21942778e-06,  -5.21942778e-06,
         5.21229121e-06,  -5.21942778e-06,   5.21942780e-06,
        -5.21942878e-06,   4.22455696e-02,  -5.21942778e-06,
        -5.21942878e-06,   5.21942778e-06,   4.22455696e-02,
         5.21942778e-06,   5.37532905e-06,   5.21942778e-06,
         5.37533445e-06,  -5.21942778e-06,  -5.21942778e-06,
         5.21942778e-06,   5.37532905e-06,   5.37532905e-06,
         5.21942778e-06,   5.21942780e-06,   5.21942780e-06,
         4.22455696e-02,   5.21942778e-06,   5.37532905e-06,
        -5.21936392e-06,   5.21942778e-06,   5.21942778e-06,
         4.22473088e-02,  -3.91020096e+00])

In [4]:
from autograd.scipy.stats import norm
import sys
def log_lik(weights, x_i, br1, br2):
    # br1 and br2 is a vector of respecive rankings.
    # We add the np.sum to vectorize the below into batches. Note that x_i can now be a matrix
    # print weights.shape, x_i.shape, br1.shape, br2.shape
    cdf = lambda y: norm.cdf( (y - np.dot(weights[:-1], np.transpose(x_i)))/weights[-1])
    return np.log(cdf(br2) - cdf(br1))

def getRating(joke):
    b = [-sys.maxint/10, -4, -2, 2, 4, sys.maxint/10]
    res = np.argmax([log_lik(wFinal, joke, b[i], b[i+1]) for i in range(len(b)-1)])
    return res + 1

In [5]:
X = np.loadtxt('X.txt')

In [6]:
ratings = [ getRating(X[i]) for i in range(len(X))]

In [7]:
def RMSE(data, ratings):
    # Calculates the RMSE
    import math
    acc = 0.0
    for row in data:
        acc += (ratings[row[0]] - row[1])**2
    return math.sqrt(acc / len(data))

In [ ]:
# Read in the data
# Each row is formatted as [User ID] [Joke ID] [Rating]
ratingsFile =  "ratings.dat"
allData = pd.read_csv(ratingsFile, sep=" ", header=None)
allData = allData.astype(int)

def splitData(R, trainPercent=0.95):
    # R is an NxM matrix where r_ij is the rating assigned by user i to jokes.
    # Here, we split the data into matrices of size ratings x 2, where the two columns are JokeID and Rating
    N,M = R.shape
    totalRatings = np.count_nonzero(R)
    testMat = np.zeros((totalRatings,2)) 
    trainMat = np.zeros((totalRatings, 2))
    trainN, testN = 0,0
    print N,M
    for j in range(M):
        tratings = float(np.count_nonzero(R[:,j]))
        inTraining = 0
        for i in range(N):
            if R[i,j] > 0:
                if inTraining <= (tratings * trainPercent):
                    inTraining += 1
                    trainMat[trainN, :] = np.array([j, R[i,j]])
                    trainN += 1
                else:
                    testMat[testN, :] = np.array([j, R[i,j]])
                    testN += 1
    # Note that the results are zero indexed! We've mofidied the joke ids so now they are 0-indexed!!!!!
    return trainMat, testMat

# drop all 0s from the data
def dropZeroRows(a):
    mask = np.all(np.isnan(a) | np.equal(a, 0), axis=1)
    return a[~mask]
R = np.loadtxt('ratings_matrix.out')

In [11]:
train, test = splitData(R)

trainE, testE = dropZeroRows(train), dropZeroRows(test)
trainE, testE = trainE.astype(int), testE.astype(int)

63978 150


In [14]:
RMSE(trainE, ratings)

2.7110850591009688